In [1]:
from selenium import webdriver
from time import sleep
from pyquery import PyQuery as pq
import requests
import time
import pandas as pd

ModuleNotFoundError: No module named 'selenium'

In [2]:
#啟動瀏覽器，放入chrome位置連結
driver = webdriver.Chrome("")

In [4]:
#打開網址
driver.get("https://www.ris.gov.tw/app/portal/3053")

In [5]:
#定義起訖日期
def sDate(sDate):
    js = 'document.getElementById("sDate").removeAttribute("readonly");'
    driver.execute_script(js)
    js_value = 'document.getElementById("sDate").value="{}"'.format(sDate)
    driver.execute_script(js_value)
def eDate(eDate):
    js = 'document.getElementById("eDate").removeAttribute("readonly");'
    driver.execute_script(js)
    js_value = 'document.getElementById("eDate").value="{}"'.format(eDate)
    driver.execute_script(js_value)

In [6]:
doorList=[]

In [7]:
for n in range(1,23):
    driver.switch_to.frame("content-frame")
    time.sleep(4)
    python_button = driver.find_element_by_xpath("/html/body/div/div[4]/div/div/form/div[2]/fieldset/div[1]/div/button")
    python_button.click()
    time.sleep(3)
    taiwan_button = driver.find_element_by_xpath("/html/body/div/div[4]/div/div/form/div/fieldset/div/map/area[{}]".format(n))
    taiwan_button.click()
    time.sleep(3)
    sDate("108-12-01")
    eDate("108-12-31")
    for i in range(2,40): #(高雄市最多區總共38個鄉鎮區)
        try:
            countySelect = driver.find_element_by_xpath("/html/body/div[1]/div[4]/div/div/form/div[2]/fieldset/div[2]/div/select/option[{}]".format(i))
            countySelect.click()
            time.sleep(2)
            searchButton=driver.find_element_by_xpath("/html/body/div[1]/div[4]/div/div/form/div[5]/button[1]").click()
            time.sleep(2)
            try:
                confirmButton1=driver.find_element_by_xpath("/html/body/div[4]/div[3]/div/button")
                confirmButton1.click()
                continue
            except:
                amountfifty=driver.find_element_by_xpath("/html/body/div[1]/div[5]/div/div/div/div[5]/div/table/tbody/tr/td[2]/table/tbody/tr/td[8]/select/option[3]").click()
                time.sleep(10)
                pageNumber=driver.find_element_by_xpath("/html/body/div[1]/div[5]/div/div/div/div[5]/div/table/tbody/tr/td[2]/table/tbody/tr/td[4]/span").text
                if int(pageNumber) ==1:
                    for column in range(0,51):
                        try:
                            countyName=driver.find_element_by_xpath("//*[@id={}]/td[2]".format(column)).text #行政區域名稱
                            date1=driver.find_element_by_xpath("//*[@id={}]/td[3]".format(column)).text #編訂日期
                            date2=driver.find_element_by_xpath("//*[@id={}]/td[4]".format(column)).text #異動日期
                            beforeAddress=driver.find_element_by_xpath("//*[@id={}]/td[5]".format(column)).text #變更前地址
                            afterAddress=driver.find_element_by_xpath("//*[@id={}]/td[6]".format(column)).text #變更後地址
                            sort=driver.find_element_by_xpath("//*[@id={}]/td[7]".format(column)).text #編訂類別
                            doorDict={}
                            doorDict["行政區域名稱"]=countyName
                            doorDict["編訂日期"]=date1
                            doorDict["異動日期"]=date2
                            doorDict["變更前地址"]=beforeAddress
                            doorDict["變更後地址"]=afterAddress
                            doorDict["編訂類別"]=sort
                            doorList.append(doorDict)
                        except:
                            pass
                if int(pageNumber)>1:
                    for nowPage in range(1,int(pageNumber)+1):
                        for column in range(0,51):
                            try:
                                countyName=driver.find_element_by_xpath("//*[@id={}]/td[2]".format(column)).text #行政區域名稱
                                date1=driver.find_element_by_xpath("//*[@id={}]/td[3]".format(column)).text #編訂日期
                                date2=driver.find_element_by_xpath("//*[@id={}]/td[4]".format(column)).text #異動日期
                                beforeAddress=driver.find_element_by_xpath("//*[@id={}]/td[5]".format(column)).text #變更前地址
                                afterAddress=driver.find_element_by_xpath("//*[@id={}]/td[6]".format(column)).text #變更後地址
                                sort=driver.find_element_by_xpath("//*[@id={}]/td[7]".format(column)).text #編訂類別
                                doorDict={}
                                doorDict["行政區域名稱"]=countyName
                                doorDict["編訂日期"]=date1
                                doorDict["異動日期"]=date2
                                doorDict["變更前地址"]=beforeAddress
                                doorDict["變更後地址"]=afterAddress
                                doorDict["編訂類別"]=sort
                                doorList.append(doorDict)
                            except:
                                pass
                        if nowPage<int(pageNumber):
                            nextPage=driver.find_element_by_xpath("/html/body/div[1]/div[5]/div/div/div/div[5]/div/table/tbody/tr/td[2]/table/tbody/tr/td[6]/span").click()
                            time.sleep(5)
                            continue
        except:
            pass
    driver.back()
    time.sleep(2)
    driver.back()
    time.sleep(2)

In [8]:
output=pd.DataFrame.from_dict(doorList)
output

,行政區域名稱,編訂日期,異動日期,變更前地址,變更後地址,編訂類別
0,臺北市松山區,民國108年12月26日,民國108年12月26日,,東昌里004鄰光復北路２４２號四樓,門牌初編
1,臺北市松山區,民國108年12月26日,民國108年12月26日,,東昌里004鄰光復北路２４２號十三樓,門牌初編
2,臺北市松山區,民國108年12月26日,民國108年12月26日,,東昌里004鄰光復北路２４２號十二樓,門牌初編
3,臺北市松山區,民國108年12月26日,民國108年12月26日,,東昌里004鄰光復北路２４２號十一樓,門牌初編
4,臺北市松山區,民國108年12月26日,民國108年12月26日,,東昌里004鄰光復北路２４２號十樓,門牌初編
...,...,...,...,...,...,...
12721,連江縣莒光鄉,民國108年12月16日,民國108年12月16日,,大坪村003鄰５５之１號三樓,門牌初編
12722,連江縣莒光鄉,民國108年12月16日,民國108年12月16日,,大坪村003鄰５５之１號二樓,門牌初編
12723,連江縣莒光鄉,民國108年12月3日,民國108年12月3日,,大坪村004鄰９０之３號,門牌初編
12724,連江縣莒光鄉,民國108年12月3日,民國108年12月3日,,西坵村001鄰１５之３號,門牌初編


In [9]:
#輸出成csv檔
output.to_csv("全台10811.csv",index=False)